# Recuperar lista de cidades no IBGE da mesorregião da Zona da Mata Mineira

## Objetivos

Recuperar dados das cidades da mesorregião da Zona da Mata Mineira


In [3]:
# curl -i -H "Accept: application/jsope: application/json" -X GET https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3112/municipios
# Link para api do IBGE
# https://servicodados.ibge.gov.br/api/docs/localidades?versao=1#api-Mesorregioes-mesorregioesMesorregiaoGet
# SIOPE
# https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2011&paginacao=-&pag=result&cod_uf=31&municipios=317130
import os, ssl
import re
import numpy as np
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup
from lxml import html

In [139]:
# Listar cidades da mesorregião da Zona da Mata Mineira
# e criar dataframe do pandas
r = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3112/municipios')
json_cities = r.json()

cities = []

for city in json_cities:
    cities.append([city['id'], city['nome']])

data = pd.DataFrame(np.array(cities), columns=['city_id','city_name'])

In [140]:
# Criar atributo com códigos compatíveis com o SIOPE/FTND
data['city_id_siope'] = data.city_id.str[:6]

In [141]:
data.head()

,city_id,city_name,city_id_siope
0,3100302,Abre Campo,310030
1,3100401,Acaiaca,310040
2,3101508,Além Paraíba,310150
3,3102050,Alto Caparaó,310205
4,3102100,Alto Rio Doce,310210


## Baixar e raspar página web com BeautifulSoup

In [66]:
# Scrape with not valid SSL certificate
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
URL = 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2011&paginacao=-&pag=result&cod_uf=31&municipios=317130'
PAGE = urllib.request.urlopen(URL).read()
soup = BeautifulSoup(PAGE)

In [18]:
#print(soup.prettify())

In [91]:
def data_from_given_city(URL, city_code, colnames, len_items):
     
    URL = '{0}{1}'.format(URL, city_code)
    PAGE = urllib.request.urlopen(URL).read()
    soup = BeautifulSoup(PAGE)

    tables = soup.find_all('table', {'class': 'table'})
    list_of_tables = []

    for i in tables:
        storeTable = i.find_all('tr')
        tabledata = []
        finaldata = []

        for row in storeTable:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            tabledata.append([ele for ele in cols if ele])

        for item in tabledata:
            if len(item) > len_items:
                for row in range(len(item)):
                    item[row] = re.sub('\xa0%','', item[row])
                    item[row] = re.sub(',','.', item[row])
                finaldata.append(item)    

        list_of_tables.append(finaldata)

    data = pd.DataFrame([item for sublist in list_of_tables for item in sublist], 
                        columns=colnames)
    
    data['city_code'] = str(city_code)
    return(data)

In [90]:
URL = 'https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2013&paginacao=%2B&pag=result&cod_uf=31&municipios=' 
city_code = 317130
URL = '{0}{1}'.format(URL, city_code)

In [99]:
data_slice_01 = data_from_given_city(URL='https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2011&paginacao=-&pag=result&cod_uf=31&municipios=',
                    city_code=317130, colnames=['id', 'var_name', 'y2008', 'y2009','y2010', 'y2011', 'y2012', 'y2013'], len_items=7)

data_slice_02 = data_from_given_city(URL='https://www.fnde.gov.br/siope/indicadoresFinanceirosEEducacionais.do?acao=PESQUISAR&anoPaginacao=2013&paginacao=%2B&pag=result&cod_uf=31&municipios=',
                    city_code=317130,
                     colnames=['id', 'var_name', 'y2014', 'y2015','y2016'],
                    len_items=4)

In [135]:
data = data_slice_01.merge(data_slice_02, on=['id', 'var_name', 'city_code'])
data.head()

,id,var_name,y2008,y2009,y2010,y2011,y2012,y2013,city_code,y2014,y2015,y2016
0,1.1,Percentual de aplicação das receitas de impost...,29.04,31.20,26.48,26.81,28.11,27.17,317130,33.18,34.49,32.99
1,1.2,Percentual de aplicação do FUNDEF ou FUNDEB na...,97.41,64.89,95.30,93.76,87.02,63.42,317130,69.91,76.51,80.23
2,1.3,Percentual de aplicação do FUNDEF ou FUNDEB em...,1.69,35.11,2.56,6.23,12.97,36.47,317130,30.09,19.25,18.30
3,1.4,Percentual das receitas do FUNDEF ou FUNDEB nã...,0.90,0.00,2.14,0.00,0.01,0.11,317130,0.00,4.24,1.47
4,2.1,Percentual dos recursos do FUNDEB aplicados na...,0.00,33.42,0.30,21.43,23.84,19.47,317130,14.37,12.42,8.10


In [136]:
data = data[ ['city_code'] + [ col for col in data.columns if col != 'city_code' ] ]

In [137]:
data.head()

,city_code,id,var_name,y2008,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016
0,317130,1.1,Percentual de aplicação das receitas de impost...,29.04,31.20,26.48,26.81,28.11,27.17,33.18,34.49,32.99
1,317130,1.2,Percentual de aplicação do FUNDEF ou FUNDEB na...,97.41,64.89,95.30,93.76,87.02,63.42,69.91,76.51,80.23
2,317130,1.3,Percentual de aplicação do FUNDEF ou FUNDEB em...,1.69,35.11,2.56,6.23,12.97,36.47,30.09,19.25,18.30
3,317130,1.4,Percentual das receitas do FUNDEF ou FUNDEB nã...,0.90,0.00,2.14,0.00,0.01,0.11,0.00,4.24,1.47
4,317130,2.1,Percentual dos recursos do FUNDEB aplicados na...,0.00,33.42,0.30,21.43,23.84,19.47,14.37,12.42,8.10
